In [ ]:
!pip install bertopic
from IPython.display import clear_output
clear_output()

In [ ]:
import bertopic
import pandas as pd
import numpy as np
from bertopic import BERTopic
from IPython.display import clear_output

In [ ]:
import os
os.environ["TOKENIZERS_PARALLELISM"] = "false"

In [ ]:
from google.colab import drive
drive.mount('/content/drive')# data uploading from google drive
data = pd.read_csv('/content/drive/MyDrive/SNA23-project-main/reddit_comments.csv') #comments file 

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
data.head()

,date,clean_text,parent_id,id,author,created_utc,num_comments,score,type,parent_author
0,26/12/2022,well yes its putting out more and more pc wish...,j0ymf78,j1rg7ow,-----fuck-----,1672086947,NaN,1,comment,omgpop
1,28/12/2022,gifgiphy,j1tx2u1,j1xj2cx,---LefeverDream---,1672197350,NaN,1,comment,SpaceNigiri
2,26/12/2022,milli vanilli,j1msww6,j1oak8n,---nom---,1672018544,NaN,1,comment,MsolProd
3,23/12/2022,that is fcking wild,zsjxbh,j1bt9y3,--DMH--,1671766049,NaN,2,comment,KNCSPROD
4,12/12/2022,thats just awesome i have some code that could...,izvrh5e,izvyzo2,--lolwutroflwaffle--,1670826291,NaN,3,comment,bilbobeenus34


In [ ]:
data.columns

Index(['date', 'clean_text', 'parent_id', 'id', 'author', 'created_utc',
       'num_comments', 'score', 'type', 'parent_author'],
      dtype='object')

In [ ]:
data['clean_text'] #column about the comments 

0        well yes its putting out more and more pc wish...
1                                                 gifgiphy
2                                           milli vanilli 
3                                      that is fcking wild
4        thats just awesome i have some code that could...
                               ...                        
49881                                       we did it boys
49882    can it be run locally though i know it was bui...
49883    im learning assembly language i had no idea ab...
49884                          ubi whats ubi sorry im dumb
49885                i wish i had the body to do onlyfans 
Name: clean_text, Length: 49886, dtype: object

In [ ]:
data.shape

(49886, 10)

In [ ]:
text=data['clean_text'].tolist() #list of the comments 

In [ ]:
text


['well yes its putting out more and more pc wishywashy answers to things it feels like some sort of pg film thats doing its best be not offend anyone it can still do some impressive thing but its gonna turn more and more into a family friendly business',
 'gifgiphy',
 'milli vanilli ',
 'that is fcking wild',
 'thats just awesome i have some code that could use some serious refactoring and general readability fixes i need to give it a go is there some limit to how much you can put in the prompt',
 'chatgpt has helped tremendously with developing my current application net but not without faults in my experience chatgpt nearly always provides me code which includes calls to methods that do not exist in some particular class requiring me to either seek an alternative method or reformulate my question as an example i was trying to modify remote windows service properties like starttype but it provided the method changestarttype in class servicecontroller that method does not exist in that

In [ ]:
len(text) #number of comments 

49886

## Pre-processing

 ### stopwords removal

In [ ]:
len(text)

49886

In [ ]:
import nltk

nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [ ]:
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords


# Tokenization of sentences
tokenized_sentences = [word_tokenize(str(frase)) for frase in text]

# stopwords removal
stopwords_set = set(stopwords.words('english'))  # Cambia 'english' se stai lavorando con un'altra lingua

frasi_senza_stopwords = [[parola for parola in frase if parola.lower() not in stopwords_set] for frase in tokenized_sentences]

# print of the sentences without stop words 
lista_finale= frasi_senza_stopwords



In [ ]:
lista_finale[:10]

[['well',
  'yes',
  'putting',
  'pc',
  'wishywashy',
  'answers',
  'things',
  'feels',
  'like',
  'sort',
  'pg',
  'film',
  'thats',
  'best',
  'offend',
  'anyone',
  'still',
  'impressive',
  'thing',
  'gon',
  'na',
  'turn',
  'family',
  'friendly',
  'business'],
 ['gifgiphy'],
 ['milli', 'vanilli'],
 ['fcking', 'wild'],
 ['thats',
  'awesome',
  'code',
  'could',
  'use',
  'serious',
  'refactoring',
  'general',
  'readability',
  'fixes',
  'need',
  'give',
  'go',
  'limit',
  'much',
  'put',
  'prompt'],
 ['chatgpt',
  'helped',
  'tremendously',
  'developing',
  'current',
  'application',
  'net',
  'without',
  'faults',
  'experience',
  'chatgpt',
  'nearly',
  'always',
  'provides',
  'code',
  'includes',
  'calls',
  'methods',
  'exist',
  'particular',
  'class',
  'requiring',
  'either',
  'seek',
  'alternative',
  'method',
  'reformulate',
  'question',
  'example',
  'trying',
  'modify',
  'remote',
  'windows',
  'service',
  'properties',


## Check for the upper case 

In [ ]:
def check_lowercase(list):
    for sentence in list:
      for token in sentence:
        if not token.islower():
              return False
      return True 




result = check_lowercase(lista_finale)
print(result)  #True if there aren't upper letters


True


 ## Check for \n

In [ ]:
def check_n(list):
    for sentence in list:
      for token in sentence:
        if  token=='\n':
              return True
      return False
result = check_n(lista_finale)
print(result)  #False if there isn't any \n

False


In [ ]:
lista_finale[:10]

[['well',
  'yes',
  'putting',
  'pc',
  'wishywashy',
  'answers',
  'things',
  'feels',
  'like',
  'sort',
  'pg',
  'film',
  'thats',
  'best',
  'offend',
  'anyone',
  'still',
  'impressive',
  'thing',
  'gon',
  'na',
  'turn',
  'family',
  'friendly',
  'business'],
 ['gifgiphy'],
 ['milli', 'vanilli'],
 ['fcking', 'wild'],
 ['thats',
  'awesome',
  'code',
  'could',
  'use',
  'serious',
  'refactoring',
  'general',
  'readability',
  'fixes',
  'need',
  'give',
  'go',
  'limit',
  'much',
  'put',
  'prompt'],
 ['chatgpt',
  'helped',
  'tremendously',
  'developing',
  'current',
  'application',
  'net',
  'without',
  'faults',
  'experience',
  'chatgpt',
  'nearly',
  'always',
  'provides',
  'code',
  'includes',
  'calls',
  'methods',
  'exist',
  'particular',
  'class',
  'requiring',
  'either',
  'seek',
  'alternative',
  'method',
  'reformulate',
  'question',
  'example',
  'trying',
  'modify',
  'remote',
  'windows',
  'service',
  'properties',


In [ ]:
len(lista_finale)

49886

In [ ]:
frasi = [' '.join(tokens) for tokens in lista_finale]#union of the sentences 


In [ ]:
frasi

['well yes putting pc wishywashy answers things feels like sort pg film thats best offend anyone still impressive thing gon na turn family friendly business',
 'gifgiphy',
 'milli vanilli',
 'fcking wild',
 'thats awesome code could use serious refactoring general readability fixes need give go limit much put prompt',
 'chatgpt helped tremendously developing current application net without faults experience chatgpt nearly always provides code includes calls methods exist particular class requiring either seek alternative method reformulate question example trying modify remote windows service properties like starttype provided method changestarttype class servicecontroller method exist class turns task required different approach pinvoke reformulating prompt instructing chatgpt thing using pinvoke able accomplish task minor code modifications said extremely useful tool programmers expect provide working copypaste code time yet',
 'matter time stabs master retribution cigarette burns',


In [ ]:
docs = frasi[:13000]


In [ ]:
len(docs)

13000

In [ ]:
model = BERTopic(embedding_model="all-mpnet-base-v2").fit(docs)
clear_output()

In [ ]:
model.get_topic_freq()

,Topic,Count
0,-1,6777
13,0,211
39,1,198
37,2,166
85,3,155
...,...,...
127,168,11
162,169,10
64,170,10
81,171,10


In [ ]:
model.visualize_topics()

In [ ]:
model.visualize_barchart()

In [ ]:
model.visualize_heatmap()

In [ ]:
model.visualize_hierarchy()

In [ ]:
model.visualize_term_rank()

In [ ]:
def compute(frasi):
    topics, probs = model.transform(frasi)
    clear_output()
    print("-----------------------------------------------------------------------------------")
    print("Topic id: ", topics)

    if 'topic_names' in dir(model):
        topic = model.topic_names[topics[0]]
        print("Topic name: ", topic)
    else:
        print("Topic names are not available.")

    print("-----------------------------------------------------------------------------------")
    print("Suggested Topic: \n")

    for i in range(len(topics)):
        print("Topic ID: ", topics[i])
        print("Topic Probability: ", probs[i])
        print("\n")


In [ ]:
compute(frasi[:13000])

Output streaming troncato alle ultime 5000 righe.
Topic ID:  32
Topic Probability:  0.8239418941287492


Topic ID:  -1
Topic Probability:  0.0


Topic ID:  54
Topic Probability:  0.6228321067736704


Topic ID:  18
Topic Probability:  0.5015192301406012


Topic ID:  -1
Topic Probability:  0.0


Topic ID:  -1
Topic Probability:  0.0


Topic ID:  -1
Topic Probability:  0.0


Topic ID:  5
Topic Probability:  0.6197149716254035


Topic ID:  10
Topic Probability:  0.8250835991484129


Topic ID:  75
Topic Probability:  0.8790786086895851


Topic ID:  75
Topic Probability:  0.9732002661753071


Topic ID:  74
Topic Probability:  0.8610166828052568


Topic ID:  17
Topic Probability:  1.0


Topic ID:  91
Topic Probability:  1.0


Topic ID:  9
Topic Probability:  1.0


Topic ID:  1
Topic Probability:  0.8643106695213015


Topic ID:  -1
Topic Probability:  0.0


Topic ID:  -1
Topic Probability:  0.0


Topic ID:  -1
Topic Probability:  0.0


Topic ID:  1
Topic Probability:  1.0


Topic ID:  128
Topi

In [ ]:
text_test=frasi[13000: 15000]

In [ ]:
prova_test=compute(text_test)

Output streaming troncato alle ultime 5000 righe.
Topic ID:  -1
Topic Probability:  0.0


Topic ID:  -1
Topic Probability:  0.0


Topic ID:  33
Topic Probability:  0.07008217881321256


Topic ID:  -1
Topic Probability:  0.0


Topic ID:  -1
Topic Probability:  0.0


Topic ID:  -1
Topic Probability:  0.0


Topic ID:  78
Topic Probability:  0.8635292410144314


Topic ID:  100
Topic Probability:  0.7000678456399629


Topic ID:  78
Topic Probability:  0.10504352598381639


Topic ID:  2
Topic Probability:  0.7887255064412908


Topic ID:  -1
Topic Probability:  0.0


Topic ID:  69
Topic Probability:  0.7059575611453968


Topic ID:  109
Topic Probability:  0.5316526520309018


Topic ID:  2
Topic Probability:  0.6984984392791308


Topic ID:  2
Topic Probability:  1.0


Topic ID:  -1
Topic Probability:  0.0


Topic ID:  -1
Topic Probability:  0.0


Topic ID:  40
Topic Probability:  0.27456742693147335


Topic ID:  64
Topic Probability:  0.9477248058717337


Topic ID:  -1
Topic Probability:  0.0


In [ ]:
model.get_topic_info()

,Topic,Count,Name,Representation,Representative_Docs
0,-1,6777,-1_chatgpt_ai_like_would,"[chatgpt, ai, like, would, im, people, one, th...",[oh gosh im sad learned dan prompt says stuff ...
1,0,211,0_content_filter_filters_banned,"[content, filter, filters, banned, nsfw, censo...",[guess correlates filters type content specifi...
2,1,198,1_story_game_player_adventure,"[story, game, player, adventure, rpgbot, write...",[prom lets play game sole protagonist story na...
3,2,166,2_openai_microsoft_conflict_partnership,"[openai, microsoft, conflict, partnership, mon...",[happened elon openai couldnt agree fundamenta...
4,3,155,3_chatgpt_chat_chadgpt_lol,"[chatgpt, chat, chadgpt, lol, chatgtp, surpris...","[chatgpt, chatgpt, chatgpt]"
...,...,...,...,...,...
169,168,11,168_links_link_retailers_zoom,"[links, link, retailers, zoom, frequency, hunc...",[couldnt either made something based text link...
170,169,10,169_legal_lawyer_boilerplate_amazed,"[legal, lawyer, boilerplate, amazed, contract,...",[im lawyer spent good deal time testing chatgp...
171,170,10,170_supposed_simpleminded_tripping_simple,"[supposed, simpleminded, tripping, simple, min...","[supposed think, simple youre simpleminded, si..."
172,171,10,171_work_doesnt_tried_pops,"[work, doesnt, tried, pops, yellow, numerous, ...","[doesnt work, doesnt work, doesnt work says ca..."


In [ ]:
model.save("topic_model", save_embedding_model=False)